In [ ]:
! pip install -U sentence-transformers


In [3]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb 
import pickle

In [4]:
config = ppb.DistilBertConfig(max_position_embeddings=512)
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel(config), ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
# cd content/

/content


In [7]:
# cd drive/My\ Drive/raw_data/  

/content/drive/My Drive/raw_data


In [8]:
with open("Onehack.pickle", 'rb') as pickle_file:
  data = pickle.load(pickle_file)

In [ ]:
# label=[]
tokens=[]
category=[]
batch_num=0
tokenout=[]

for batch in range(0,len(data)-100,100):  
  print(batch)
  batch_num+=1
  # label=[]
  tokens=[]
  # category=[]
  for topic in range(batch,batch+100):

    # label.append(slug[topic]['slug'])
    tokens.append(tokenizer.encode(data[topic][-1], max_length=512,pad_to_max_length=True,truncation=True,add_special_tokens=True))
    category.append(data[topic][1])
    if (topic == batch+99):

      tokenout.append(tokens)
      input_ids = torch.tensor(np.array(tokens))

      with torch.no_grad():
        last_hidden_states = model(input_ids)

      if (batch_num==1):
        features = last_hidden_states[0][:,0,:].numpy()
      
      else:
        features = np.concatenate((features,last_hidden_states[0][:,0,:].numpy()),axis=0)
      
      print("Done with Batch:",batch_num)
      
with open("Onehack_features.pickle", "wb") as output_file:
  pickle.dump(features, output_file)

#label = np.asarray(label).reshape(-1,1)
category = np.asarray(category).reshape(-1,1)
tokens = np.asarray(tokens)

pd.DataFrame(tokenout).to_csv("Onehack_tokens.csv")
#pd.DataFrame(label).to_csv("bnzlabels.csv")
pd.DataFrame(category).to_csv("Onehack_category.csv")
      

In [13]:
from scipy.spatial.distance import cdist
Y = cdist(features, features, 'cosine')

In [14]:
# cd ..

/content/drive/My Drive


In [15]:
# cd raw_data/

/content/drive/My Drive/raw_data


In [16]:
with open("Onehack.pickle", 'rb') as pickle_file:
  text = pickle.load(pickle_file)

In [17]:
def recommend():
  val = int( input("Enter a number between 0 and 8400:"))
  print('')
  print("The article you've chosen:",data[val])

  similar_articles =  np.argsort(Y[val][:])[:11]
  print('')
  print("Few Articles you might like reading:")
  print('')
  for i in similar_articles[1:]:
    print(data[i])

In [ ]:
recommend()

In [19]:
len(data)

8446